# ・ライブラリのインポート

In [11]:
!pip install hyperopt

     |████████████████████████████████| 965 kB 4.8 MB/s eta 0:00:01


In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# ・データの読み込み

In [13]:
train = pd.read_csv('/Users/markun/SIGNATE/train.csv')
test = pd.read_csv('/Users/markun/SIGNATE/test.csv')

In [14]:
test

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,398,0,126,80,0,0,40.096264,0.822517,21
1,3833,3,88,60,20,0,39.810590,0.204331,22
2,4836,3,114,76,0,0,33.198760,0.521011,21
3,4572,1,146,74,0,0,26.890259,0.504950,38
4,636,1,123,90,26,140,40.270088,0.800513,28
...,...,...,...,...,...,...,...,...,...
1995,3138,4,150,60,0,0,39.385785,0.175051,26
1996,191,6,153,88,0,0,38.614204,0.509367,28
1997,3294,7,106,78,0,0,51.678147,0.728404,29
1998,3073,2,101,70,0,0,40.271989,0.316558,26


In [15]:
train

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,200,9,125,74,0,0,28.536910,0.444902,45,1
1,3832,4,109,80,0,0,28.047673,0.238243,22,0
2,4927,4,88,78,39,0,52.371341,0.279471,26,0
3,4088,9,125,74,0,0,40.062688,0.203922,45,0
4,3644,5,107,78,44,284,52.935068,0.284959,45,1
...,...,...,...,...,...,...,...,...,...,...
2995,4931,4,88,74,17,0,33.848723,0.171073,23,0
2996,3264,0,144,88,0,0,26.846832,0.259957,21,1
2997,1653,6,117,96,36,0,28.101646,0.716126,22,1
2998,2607,2,113,74,0,0,33.079021,0.266179,38,1


# ・目的変数を除外

In [16]:
train_x = train.drop(['Outcome'], axis = 1)
train_x 

,index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,200,9,125,74,0,0,28.536910,0.444902,45
1,3832,4,109,80,0,0,28.047673,0.238243,22
2,4927,4,88,78,39,0,52.371341,0.279471,26
3,4088,9,125,74,0,0,40.062688,0.203922,45
4,3644,5,107,78,44,284,52.935068,0.284959,45
...,...,...,...,...,...,...,...,...,...
2995,4931,4,88,74,17,0,33.848723,0.171073,23
2996,3264,0,144,88,0,0,26.846832,0.259957,21
2997,1653,6,117,96,36,0,28.101646,0.716126,22
2998,2607,2,113,74,0,0,33.079021,0.266179,38


# ・目的変数

In [17]:
train_y = train['Outcome']
train_y

0       1
1       0
2       0
3       0
4       1
       ..
2995    0
2996    1
2997    1
2998    1
2999    0
Name: Outcome, Length: 3000, dtype: int64

# ・訓練データを訓練用と検証用に分割する

In [18]:
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# ・XGB

In [26]:
class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        dvalid = xgb.DMatrix(va_x, label=va_y)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.model = xgb.train(params, dtrain, num_round, evals=watchlist)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

NameError: name 'predict' is not defined

In [20]:
import numpy as np
from hyperopt import hp

# -----------------------------------
# xgboostのパラメータ空間の例
# -----------------------------------

# ベースラインのパラメータ
params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'eta': 0.1,
    'gamma': 0.0,
    'alpha': 0.0,
    'lambda': 1.0,
    'min_child_weight': 1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 71,
}

# パラメータの探索範囲
param_space = {
    'min_child_weight': hp.loguniform('min_child_weight', np.log(0.1), np.log(10)),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'subsample': hp.quniform('subsample', 0.6, 0.95, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.95, 0.05),
    'gamma': hp.loguniform('gamma', np.log(1e-8), np.log(1.0)),
    # 余裕があればalpha, lambdaも調整する
    # 'alpha' : hp.loguniform('alpha', np.log(1e-8), np.log(1.0)),
    # 'lambda' : hp.loguniform('lambda', np.log(1e-6), np.log(10.0)),
}

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import log_loss


def score(params):
    # パラメータを与えたときに最小化する評価指標を指定する
    # 具体的には、モデルにパラメータを指定して学習・予測させた場合のスコアを返すようにする

    # max_depthの型を整数型に修正する
    params['max_depth'] = int(params['max_depth'])

    # Modelクラスを定義しているものとする
    # Modelクラスは、fitで学習し、predictで予測値の確率を出力する
    model = Model(params)
    model.fit(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')

    # 情報を記録しておく
    history.append((params, score))

    return {'loss': score, 'status': STATUS_OK}


# 探索するパラメータの空間を指定する
space = {
    'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
    'max_depth': hp.quniform('max_depth', 3, 9, 1),
    'gamma': hp.quniform('gamma', 0, 0.4, 0.1),
}
max_evals = 10
trials = Trials()
history = []
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

# 記録した情報からパラメータとスコアを出力する
# （trialsからも情報が取得できるが、パラメータの取得がやや行いづらいため）
history = sorted(history, key=lambda tpl: tpl[1])
best = history[0]
print(f'best params:{best[0]}, score:{best[1]:.4f}')

[21:47:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.19422	eval-error:0.22267            

[1]	train-error:0.17911	eval-error:0.21200            

[2]	train-error:0.17733	eval-error:0.20533            

[3]	train-error:0.16356	eval-error:0.20000            

[4]	train-error:0.15511	eval-error:0.20000            

[5]	train-error:0.15022	eval-error:0.20000            

[6]	train-error:0.14578	eval-error:0.19867            

[7]	train-error:0.14533	eval-error:0.19867            

[8]	train-error:0.13244	eval-error:0.20400            

[9]	train-error:0.13067	eval-error:0.20000            

params: {'gamma': 0.1, 'max_depth': 6, 'min_child_weight': 4.0}, logloss: 0.4358
[21

[5]	train-error:0.18133	eval-error:0.19733                                       

[6]	train-error:0.17378	eval-error:0.19200                                       

[7]	train-error:0.16800	eval-error:0.18800                                       

[8]	train-error:0.16578	eval-error:0.18800                                       

[9]	train-error:0.16489	eval-error:0.18533                                       

params: {'gamma': 0.1, 'max_depth': 4, 'min_child_weight': 2.0}, logloss: 0.4430 
[21:47:29] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516:         
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.20044	eval-error:0.22800                                       

[1]	train-error:0.19156	eval-error:0.19867              

#  ・テストデータに対する予測

In [30]:
pred = Model(xgb.DMatrix(test))
pred = np.where(pred > 0.5, 1,0)
pred

TypeError: '>' not supported between instances of 'Model' and 'float'

# ・提出用ファイルの作成

In [ ]:
submission6 = pd.DataFrame({
    'index':test['index'],
    'Outcome':prediction, 
})
submission6.to_csv('/Users/markun/SIGNATE/submission6.csv', header = False, index = False )

In [ ]:
submission5.head()